In [1]:
!python -m pip install semantic-kernel==0.9.4b1


[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import semantic_kernel as sk

kernel = sk.Kernel()

In [2]:
from services import Service

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = Service.AzureOpenAI

In [3]:
kernel = sk.Kernel()

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    api_key, org_id = sk.openai_settings_from_dot_env()
    service_id = "gpt4-32k"
    kernel.add_service(
        OpenAIChatCompletion(service_id=service_id, ai_model_id="gpt4-32k", api_key=api_key, org_id=org_id),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    service_id = "aoai_chat_completion"
    kernel.add_service(
        AzureChatCompletion(service_id=service_id, deployment_name=deployment, endpoint=endpoint, api_key=api_key),
    )

In [4]:
# note: using plugins from the samples folder
plugins_directory = "plugins"

dataFunctions = kernel.import_plugin_from_prompt_directory(plugins_directory, "DataPlugin")

descriptorFunction = dataFunctions["DatabaseDescriptor"]

In [5]:
def read_data_schema_from_file(file_path):
    with open(file_path, 'r') as file:
        data_schema = file.read()
    return data_schema

file_path = "data_schema.txt"
data_schema = read_data_schema_from_file(file_path)


In [6]:
query = "what are best performing sales products in total?"

result = await kernel.invoke(descriptorFunction, sk.KernelArguments(data_schema=data_schema, query= query))
print(result)

Variable `Symbols.VAR_PREFIX: save_plot_to_disk` not found in the KernelArguments


<sql>
SELECT p.product_name, SUM(oi.quantity) AS total_quantity_sold
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
JOIN orders o ON oi.order_id = o.order_id
WHERE strftime('%Y', o.order_date) = '2024'
GROUP BY p.product_name
ORDER BY total_quantity_sold DESC
LIMIT 10;
</sql>


In [7]:
import re

def parse_text_between_tags(text, start_tag, end_tag):
    pattern = rf"{re.escape(start_tag)}(.*?){re.escape(end_tag)}"
    matches = re.findall(pattern, text, re.DOTALL)
    return matches


In [8]:
# Extract the string data from the result object
result_string = result.data if hasattr(result, 'data') else str(result)

matches = parse_text_between_tags(result_string,"<sql>", "</sql>")
print(matches[0])


SELECT p.product_name, SUM(oi.quantity) AS total_quantity_sold
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
JOIN orders o ON oi.order_id = o.order_id
WHERE strftime('%Y', o.order_date) = '2024'
GROUP BY p.product_name
ORDER BY total_quantity_sold DESC
LIMIT 10;



In [9]:
import sqlite3
import pandas as pd
import os

def run_sql_query( query):
    db_conn = os.getenv("DB_CONNECTION_STRING")
    print(db_conn)
    conn = sqlite3.connect("BikeStore.db")
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df


In [10]:
df = run_sql_query(matches[0])
df.head()

BikeStore.db


,product_name,total_quantity_sold


In [12]:
query ="what are best performing sales products in total?"

result = await kernel.invoke(descriptorFunction, sk.KernelArguments(data_schema=data_schema, query= query))
print(result)

Variable `Symbols.VAR_PREFIX: save_plot_to_disk` not found in the KernelArguments


```sql
<sql>
SELECT p.product_name, SUM(oi.quantity * oi.list_price * (1 - oi.discount)) AS total_sales
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
GROUP BY p.product_name
ORDER BY total_sales DESC
LIMIT 10;
</sql>
```

```python
<python>
import pandas as pd
import matplotlib.pyplot as plt

# Assuming df is already defined and contains the data from the SQL query
df = pd.DataFrame({
    'product_name': ['Product A', 'Product B', 'Product C'],  # Example data
    'total_sales': [1000, 1500, 2000]  # Example data
})

plt.figure(figsize=(10, 6))
plt.bar(df['product_name'], df['total_sales'], color='skyblue')
plt.xlabel('Product Name')
plt.ylabel('Total Sales')
plt.title('Top 10 Best Performing Sales Products')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
</python>
```
